<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/Notebooks/Frozen_RAG_%26_HyDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install  transformers
!pip install langchain
!pip install -U langchain-community
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install LangChain -q
!pip install langchain transformers
!pip install chromadb
!pip install sentence-transformers
!pip install rank-bm25 sentence-transformers
!pip uninstall nltk -y
!pip install nltk
!pip install openai
!pip install groq
!pip install langchain-groq
!pip install gradio

In [3]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
import re
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import chromadb
import os
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage  # Import HumanMessage
import gradio as gr
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document


# Load TXT

In [3]:
data_loader = DirectoryLoader("/content/Data",
                             glob="*.txt",
                             show_progress=True)


In [4]:
text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size based on content length
    chunk_overlap=0,  # No overlap needed
    separators=["Page number"]  # Split at answer or next question
)

In [5]:
# Load documents from the specified directory
data = data_loader.load_and_split(text_splitter=text_splitter_recursive) # Load documents

100%|██████████| 10/10 [02:18<00:00, 13.83s/it]


In [6]:
print(f"Documents : {len(data)}.")

Documents : 3391.


In [7]:
# Remove "Page number " from each chunk using regex
for i in range(len(data)):
  data[i].page_content = re.sub(r"Page number: \d+", "", data[i].page_content)

# Embeddding

In [ ]:

# Load the base model
base_model = SentenceTransformer('intfloat/multilingual-e5-large')

In [6]:



path = "/content/DB"
client = chromadb.PersistentClient(path=path)

db_file = client.get_or_create_collection(name='Book',
                                          metadata={"hnsw:space": "cosine"})

In [9]:
def embed(texts):
    return base_model.encode(texts)

i = 0
for chunk in data:
    embeddings = embed([chunk.page_content])  # Get embeddings using the base model
    db_file.add(
        documents=[chunk.page_content],
        metadatas=[chunk.metadata],
        ids=[f"chunk_{i}"],
        embeddings=embeddings  # Pass the embeddings directly
    )
    i += 1

In [ ]:
db_file.get(ids=["chunk_2"])

In [8]:
def retrieve_documents_embeddings(query_embedding, k=10):
    query_embedding_list = query_embedding.tolist()

    results = db_file.query(
        query_embeddings=[query_embedding_list],

        n_results=k)
    return results

# LLM

In [9]:


# Set the GROQ API Key as an environment variable
os.environ["GROQ_API_KEY"] = "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"

# Initialize the ChatGroq model
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),  # Retrieve the API key from the environment variable
    model="llama-3.1-70b-versatile",
    temperature=0.7,
    max_tokens=500,
)

# Create a list of messages using the HumanMessage class
messages = [
    HumanMessage(content="Hello")
]

# Sending a message to the model
response = llm.invoke(messages)  # Use invoke instead of call

# Handle streaming response
for chunk in response:
    # Assuming the structure is (content, metadata), modify according to actual structure
    content = chunk[0] if isinstance(chunk, tuple) else chunk
    print(content or "", end="")


contentadditional_kwargsresponse_metadatatypenameidexampletool_callsinvalid_tool_callsusage_metadata

In [10]:
print(response.content)

Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?


In [11]:
#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

# Frozen RAG

In [15]:
qna_template = "\n".join([
    "Act as a knowledgeable law professor. Analyze the provided legal context and respond to the subsequent question with thoroughness and clarity. If the information needed to answer the question is not present in the context, respond with 'NO ANSWER IS AVAILABLE.'",
    "give the answer in details with Arabic accent ",

    "### Context",
    "{context}",

    "### Question",
    "{question}",

    "### Answer:",
])

qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

stuff_chain = load_qa_chain(llm, chain_type="stuff", prompt=qna_prompt)


In [16]:
# Sample query
query = ''''

ما أهمية التنفيذ الجبري في قانون المرافعات؟

'''

# Generate the embedding for the query
query_embedding = base_model.encode(query)

# Perform similarity search in ChromaDB
similar_documents = retrieve_documents_embeddings(query_embedding, k=7)



In [17]:
documents = similar_documents['documents'][0]
metadatas = similar_documents['metadatas'][0]

from langchain.schema import Document
formatted_documents = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(documents, metadatas)
]

In [ ]:
# Use the stuff chain to generate the answer
answer = stuff_chain(
    {
        "input_documents": formatted_documents,  # Pass the search results directly
        "question": query  # Pass the original query
    },
    return_only_outputs=True,  # Return only the output from the chain
)

# Print the answer
print("Generated Answer:", answer)


In [ ]:
# Get the output text from the answer
output_text = answer['output_text']

# Print the formatted answer
print("Generated Answer:")
print("=" * 50)  # Separator line
print(output_text)


# Gradio Frozen RAG

In [ ]:
# prompt: make the above gradio code also print the metadats

def generate_answer(query):
    # Generate the embedding for the query
    query_embedding = base_model.encode(query)

    # Perform similarity search in ChromaDB
    similar_documents = retrieve_documents_embeddings(query_embedding, k=7)

    documents = similar_documents['documents'][0]
    metadatas = similar_documents['metadatas'][0]

    formatted_documents = [
        Document(page_content=doc, metadata=meta)
        for doc, meta in zip(documents, metadatas)
    ]
    # Use the stuff chain to generate the answer
    answer = stuff_chain(
        {
            "input_documents": formatted_documents,  # Pass the search results directly
            "question": query  # Pass the original query
        },
        return_only_outputs=True,  # Return only the output from the chain
    )

    # Get the output text from the answer
    output_text = answer['output_text']

    # Prepare similar documents for display
    similar_docs_str = "\n".join([f"**Document {i+1}:**\n{doc}\n**Metadata:** {meta}" for i, (doc, meta) in enumerate(zip(documents, metadatas))])

    return output_text, similar_docs_str



iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(lines=5, placeholder="Enter your query here..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Similar Documents with Metadata")
    ],

        examples=[
        "ما أهمية التنفيذ الجبري في قانون المرافعات؟",
        "ما هي قواعد الاختصاص القضائي في القانون المصري؟",
    ],
    title="Muffakir",
    description="Ask your questions"
)

iface.launch(share=True)

# Hypothetical Document Embeddings


In [61]:
# Sample query
query_hyde = ''''

ما أهمية التنفيذ الجبري في قانون المرافعات؟

'''

In [62]:
# Hypothetical Document Embeddings




# HyDE document genration prompt
template_hyde = "\n".join([
    "Act as a Law professor from egypt. Analyze the provided law question and respond to the subsequent question with thoroughness and clarity.",
    "give the answer in details in arabic only ",


    "### Question",
    "{question}",

    "### Answer:",
])
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

# generating a hypothetical document based on the user input
hypothetical_document = llm.invoke(prompt_hyde.format(question=query_hyde))


In [63]:


# Generate the embedding for the query
hypothetical_document_embedding = base_model.encode(hypothetical_document.content)

# Perform similarity search in ChromaDB
similar_documents_hyde = retrieve_documents_embeddings(hypothetical_document_embedding, k=5)



documents_hyde = similar_documents_hyde['documents'][0]
metadatas_hyde = similar_documents_hyde['metadatas'][0]

from langchain.schema import Document
formatted_documents_hyde = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(documents_hyde, metadatas_hyde)
]


In [ ]:
answer = stuff_chain(
        {
            "input_documents": formatted_documents_hyde ,
            "question": query
        },
        return_only_outputs=True
    )


# Get the output text from the answer
output_text = answer['output_text']

# Print the formatted answer
print("Generated Answer:")
print("=" * 50)  # Separator line
print(output_text)


# Gradio HyDE

In [ ]:

# HyDE document generation prompt
template_hyde = "\n".join([
    "Act as a Law professor from Egypt. Analyze the provided law question and respond to the subsequent question with thoroughness and clarity.",
    "give the answer in details in arabic only.",

    "### Question",
    "{question}",

    "### Answer:",
])
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

# Function to handle chatbot interaction
def law_chatbot(query):
    try:
        # Generate hypothetical document
        hypothetical_document = llm.invoke(prompt_hyde.format(question=query))

        # Generate embedding for the hypothetical document
        hypothetical_document_embedding = base_model.encode(hypothetical_document.content)

        # Perform similarity search in ChromaDB
        similar_documents_hyde = retrieve_documents_embeddings(hypothetical_document_embedding, k=5)

        documents_hyde = similar_documents_hyde['documents'][0]
        metadatas_hyde = similar_documents_hyde['metadatas'][0]

        # Format documents for input into the stuff chain
        formatted_documents_hyde = [
            Document(page_content=doc, metadata=meta)
            for doc, meta in zip(documents_hyde, metadatas_hyde)
        ]

        # Use the chain to generate the answer
        answer = stuff_chain(
            {
                "input_documents": formatted_documents_hyde,
                "question": query
            },
            return_only_outputs=True
        )

        # Extract the output text
        output_text = answer['output_text']

        # Prepare similar documents and metadata for separate display
        documents_display = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(documents_hyde)])
        metadata_display = "\n\n".join([f"Metadata {i+1}:\n{meta}" for i, meta in enumerate(metadatas_hyde)])

        return output_text, documents_display, metadata_display

    except Exception as e:
        return f"An error occurred: {str(e)}", "", ""

# Gradio interface with multiple output boxes
iface = gr.Interface(
    fn=law_chatbot,
    inputs="text",
    outputs=[
        gr.Textbox(label="Chatbot Answer", lines=10),
        gr.Textbox(label="Similar Documents", lines=10),
        gr.Textbox(label="Metadata", lines=10),
    ],


        examples=[
        "ما أهمية التنفيذ الجبري في قانون المرافعات؟",
        "ما هي قواعد الاختصاص القضائي في القانون المصري؟",
    ],

    title="Muffakir HyDE ",
    description="Ask any question",

)

# Launch the chatbot
iface.launch()
